In [22]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

import tensorflow as tf
import pandas as pd

In [2]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_cortadas"

Obtenemos de las carpetas originales las fotos

In [3]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


In [4]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" #lo tenemos en memoria pero lo leemos nuevamente por si separamos el codigo
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
from funciones import transformar_imagen_pca

imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)

imagenes_pca_reducidas.shape
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print(cantidad_personas) 
cantidad_fotos = len(image_person)
cantidad_fotos


imagenes_pca_reducidas[0]
X=imagenes_pca_reducidas
X.shape
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
salida_lista
salida_lista.shape
X = imagenes_pca_reducidas
Y = salida_lista

# Normalización de las entradas
"""mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
"""
# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 1100


# Semilla para reproducibilidad
np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

[[ 7.22160860e+00 -7.80849051e+00 -2.07612021e-01 ... -7.16897218e-01
  -2.28025618e-01 -3.38966655e-01]
 [-7.84286624e+00 -1.12024841e+01  4.89598300e-01 ... -7.43509551e-01
  -6.18654161e-01  2.88035939e-01]
 [-6.55630242e+00 -7.80088794e+00 -1.49179444e+00 ... -1.37140978e+00
  -2.96450608e-01  2.05965925e-02]
 ...
 [ 3.55657307e+00  6.49301206e-01 -6.23891347e+00 ... -6.32500626e-02
   3.18870940e-01 -6.26304974e-01]
 [-2.65755973e+00 -4.44534213e-03 -8.82886566e+00 ...  2.92722209e-01
  -9.78650328e-02 -2.65089331e-01]
 [-2.62864833e+00  8.66772857e-01 -8.11920552e+00 ...  1.20877517e-01
  -1.40180893e-02 -1.42284246e-01]]
19


c:\Users\natal\.conda\envs\envtp\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
print(X_train.shape)
X_test.shape

(964, 60)


(242, 60)

In [6]:
print(Y_train.shape)
Y_test.shape

(964, 19)


(242, 19)

In [8]:

def create_model(optimizer='adam', activation='relu', neurons=128, **kwargs):
    

    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_componentes,)),  # Capa de entrada con 60 características - Cantidad de componentes PCA
    tf.keras.layers.Dense(neurons, activation='relu'),  # Capa oculta con 128 neuronas y función de activación ReLu (trasmite la infoamción generada por la combinación lineal de los pesos y las entredas, especificamente ReLu permite la propagación hacia atrás efectiva)
                                                    # ReLu genera una salida igual a cero cuando la entrada (z) sea negativa, y una salida igual a la entrada cuando está última es positiva. 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(cantidad_personas, activation='softmax') # Capa de salida con 18 neuronas y función de activación Softmax (La función calcula la distribución de probabilidades del evento sobre "n" eventos diferentes)
                                                    # Convierte los datos en probabilidades y la suma de las probabilidades da como resultado 1.

    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [9]:
# Creamos clasificador

model = KerasClassifier(model=create_model, verbose=0)

In [10]:
# Definir hiperparámetros

param_grid = {
    'model__optimizer': ['adam', 'sgd'],
    'model__activation': ['relu', 'sigmoid'],
    'model__neurons': [64, 128, 256,512], #neuronas de la capa oculta
    'epochs': [20, 30, 60,70]  # Añadimos el número de épocas como hiperparámetro
}

In [13]:
# Buscamos Hiperparámetros

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_train, Y_train)

In [14]:
# Mejores Hiperparámetros

print("Mejores hiperparámetros:", grid_result.best_params_)


Mejores hiperparámetros: {'epochs': 60, 'model__activation': 'relu', 'model__neurons': 512, 'model__optimizer': 'adam'}


In [15]:
grid_result

GridSearchCV(cv=3,
             estimator=KerasClassifier(model=<function create_model at 0x000002225AEEC2C0>, verbose=0),
             param_grid={'epochs': [20, 30, 60, 70],
                         'model__activation': ['relu', 'sigmoid'],
                         'model__neurons': [64, 128, 256, 512],
                         'model__optimizer': ['adam', 'sgd']})

In [16]:
# OBTENER EL MEJOR MODELO

best_model = grid_result.best_estimator_

In [17]:
# EVALUAR EL MODELO EN LOS DATOS DE PRUEBA

score = best_model.score(X_test, Y_test)
print("Precisión en los datos de prueba:", score)

#best_model.score(X_test, y_test) evalúa el mejor modelo en el conjunto de prueba, devolviendo la precisión del modelo.

Precisión en los datos de prueba: 0.9793388429752066


In [18]:
# PRECISIÓN DE LOS DATOS DE PRUEBA

from sklearn.metrics import classification_report, confusion_matrix

# Predecir etiquetas para el conjunto de prueba
y_pred = best_model.predict(X_test)


# obtener el de mayor probabiliad para sacar los valores predichos sobre los X de testeo
y_pred_max = np.argmax(y_pred, axis=1)

# los valores reales de test
y_test_true = np.argmax(Y_test, axis=1)

# Imprimir el reporte de clasificación
print(classification_report(y_test_true, y_pred_max))

# Imprimir la matriz de confusión
print(confusion_matrix(y_test_true, y_pred_max))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00        18
           6       1.00      1.00      1.00         9
           7       0.94      1.00      0.97        16
           8       1.00      1.00      1.00         7
           9       1.00      0.82      0.90        17
          10       1.00      1.00      1.00        10
          11       0.87      1.00      0.93        20
          12       1.00      1.00      1.00        13
          13       1.00      1.00      1.00        10
          14       1.00      1.00      1.00        15
          15       1.00      1.00      1.00         7
          16       0.93      1.00      0.97        14
          17       1.00    

In [19]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
# Convertir índices de clases a nombres de clases
y_test_names = label_binarizer.inverse_transform(Y_test) #reales de test
y_pred_names = label_binarizer.inverse_transform(y_pred)


# Crear DataFrame
tabla_resultados = pd.DataFrame({
    'y_predicho': y_pred_names,
    'y_testeo': y_test_names
})
tabla_resultados['Predicción correcta'] = tabla_resultados['y_predicho'] == tabla_resultados['y_testeo']
tabla_resultados

,y_predicho,y_testeo,Predicción correcta
0,Florencia,Florencia,True
1,Lautaro,Lautaro,True
2,Victorio,Victorio,True
3,Paola,Paola,True
4,Franco S,Franco S,True
...,...,...,...
237,Franco S,Franco S,True
238,Carlos,Carlos,True
239,Natalia,Natalia,True
240,Carlos,Carlos,True


In [24]:
#tabla_resultados['Predicción correcta']==false
np.sum(tabla_resultados['Predicción correcta'])/len(tabla_resultados)

0.9793388429752066

In [25]:
filtro_falsos = tabla_resultados[tabla_resultados['Predicción correcta'] == False]
print(filtro_falsos)

    y_predicho  y_testeo  Predicción correcta
69      Noelia      Abel                False
149    Lautaro   Joaquin                False
162    Lautaro   Joaquin                False
177    Lautaro   Joaquin                False
199     Gerard  Victorio                False
